In this notebook, we will use the node2vecs package which was modified in "Final - Modding Sadamori's Torch N2V.ipynb" in the previously timestamped folder. We modified the loss functions to use cosine and Euclidean distances apart from just dot similarity. The autograd function was used to verify our manual jacobian calculations. 
We now generate embeddings through this modified node2vec to suit our needs later. We want to use the embeddings to run clustering with different distance metrics to see if there's something interesting.


I want to use another set of networks because of some weird aberrations that were happening in low mixing limits in the previous run.

In [ ]:
import os
import numpy as np
import scipy.sparse as sp
import pandas as pd
import pickle
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from node2vecs import TorchNode2Vec

def process_task(task):
    """
    Process a single task: load the network and community table, train embeddings using three similarity metrics,
    and save embeddings, network, and community table to the output directory.
    
    task: tuple (k, run, mu, task_idx)
    """
    k, run, mu, task_idx = task
    # Alternate GPU devices: tasks with even task_idx use cuda:0, odd use cuda:1
    device = f"cuda:{task_idx % 4}"
    
    # Define input and output directories for this k value
    input_base = f"/nobackup/gogandhi/alt_means_sans_k/data/experiment_mu_change_10000_{k}_3.0_minc50"
    output_base = f"/nobackup/gogandhi/alt_means_sans_k/data/experiment_n2v_metric_change_10000_{k}_3.0_minc50"
    
    input_run_dir = os.path.join(input_base, f"Run_{run}")
    output_run_dir = os.path.join(output_base, f"Run_{run}")
    os.makedirs(output_run_dir, exist_ok=True)
    
    mu_str = f"{mu}"
    net_filename = f"net_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu_str}_k_{k}_mincomm_50.npz"
    comm_filename = f"community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu_str}_k_{k}_mincomm_50.npz"
    
    net_path = os.path.join(input_run_dir, net_filename)
    comm_path = os.path.join(input_run_dir, comm_filename)
    
    if not os.path.exists(net_path):
        print(f"[Task {task_idx}] Network file not found: {net_path}")
        return
    if not os.path.exists(comm_path):
        print(f"[Task {task_idx}] Community file not found: {comm_path}")
        return
    
    # Load the network as a scipy sparse matrix
    try:
        A = sp.load_npz(net_path)
    except Exception as e:
        print(f"[Task {task_idx}] Error loading network file {net_path}: {e}")
        return
    
    # Load the community table; assume it's saved in an npz file
    try:
        community_table = pd.read_csv(comm_path)
        
    except Exception as e:
        print(f"[Task {task_idx}] Error loading community file {comm_path}: {e}")
        return
    
    # Define the similarity metrics to test
    similarity_measures = ["dot", "euclidean", "cosine"]
    embeddings_dict = {}
    
    for sim in similarity_measures:
        print(f"[Task {task_idx}] Training model using {sim} similarity for Run {run}, mu {mu_str}, k {k} on {device}...")
        try:
            model = TorchNode2Vec(
                vector_size=64,
                similarity_metric=sim,
                device=device,
                num_workers=1
            )
            model.fit(A)
            emb = model.transform()
            embeddings_dict[sim] = emb
        except Exception as e:
            print(f"[Task {task_idx}] Error training {sim} model for Run {run}, mu {mu_str}, k {k} on {device}: {e}")
            continue
    
    # Save the embeddings dictionary as a pickle file
    out_filename = f"embeddings_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu_str}_k_{k}_mincomm_50.pkl"
    out_path = os.path.join(output_run_dir, out_filename)
    try:
        with open(out_path, "wb") as f:
            pickle.dump(embeddings_dict, f)
        print(f"[Task {task_idx}] Saved embeddings to {out_path}")
    except Exception as e:
        print(f"[Task {task_idx}] Error saving embeddings to {out_path}: {e}")
    
    # Save the network to the output folder
    network_out_path = os.path.join(output_run_dir, net_filename)
    try:
        sp.save_npz(network_out_path, A)
        print(f"[Task {task_idx}] Saved network to {network_out_path}")
    except Exception as e:
        print(f"[Task {task_idx}] Error saving network to {network_out_path}: {e}")
    
    # Save the community table as a CSV file to the output folder
    community_out_path = os.path.join(output_run_dir, f"community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu_str}_k_{k}_mincomm_50.csv")
    try:
        if not isinstance(community_table, pd.DataFrame):
            community_table = pd.DataFrame(community_table)
        community_table.to_csv(community_out_path, index=False)
        print(f"[Task {task_idx}] Saved community table to {community_out_path}")
    except Exception as e:
        print(f"[Task {task_idx}] Error saving community table to {community_out_path}: {e}")

# Process k values sequentially while parallelizing runs and mu-values
for k in [10,50]:
    tasks = []
    mu_values = [round(x, 2) for x in np.arange(0.05, 1.01, 0.05)]
    task_idx = 0
    for run in range(1, 11):
        for mu in mu_values:
            tasks.append((k, run, mu, task_idx))
            task_idx += 1
    
    # Use a ProcessPoolExecutor to run tasks in parallel; adjust max_workers as needed
    with ProcessPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_task, task) for task in tasks]
        for _ in tqdm(as_completed(futures), total=len(futures), desc=f"Processing k={k}"):
            pass

Processing k=10:   0%|                                                                                  | 0/200 [00:00<?, ?it/s]

[Task 1] Training model using dot similarity for Run 1, mu 0.1, k 10 on cuda:1...[Task 5] Training model using dot similarity for Run 1, mu 0.3, k 10 on cuda:1...
[Task 8] Training model using dot similarity for Run 1, mu 0.45, k 10 on cuda:0...

[Task 0] Training model using dot similarity for Run 1, mu 0.05, k 10 on cuda:0...
[Task 3] Training model using dot similarity for Run 1, mu 0.2, k 10 on cuda:3...[Task 4] Training model using dot similarity for Run 1, mu 0.25, k 10 on cuda:0...[Task 2] Training model using dot similarity for Run 1, mu 0.15, k 10 on cuda:2...
[Task 6] Training model using dot similarity for Run 1, mu 0.35, k 10 on cuda:2...
[Task 7] Training model using dot similarity for Run 1, mu 0.4, k 10 on cuda:3...


[Task 9] Training model using dot similarity for Run 1, mu 0.5, k 10 on cuda:1...


  5%|███▍                                                                        | 1425/31250 [00:15<05:05, 97.73it/s, loss=1.3]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  9%|███████                                                                   | 2967/31250 [00:31<04:39, 101.23it/s, loss=1.14]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 10%|███████                                                                   | 2980/31250 [0

 35%|█████████████████████████▊                                               | 11073/31250 [01:52<03:28, 96.84it/s, loss=0.967]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 34%|█████████████████████████▏                                                | 10612/31250 [01:59<03:41, 93.04it/s, loss=1.21]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 46%|█████████████████████████████████▊                                       | 14482/31250 [0

[Task 3] Training model using euclidean similarity for Run 1, mu 0.2, k 10 on cuda:3...

...

[Task 91] Saved community table to /nobackup/gogandhi/alt_means_sans_k/data/experiment_n2v_metric_change_10000_10_3.0_minc50/Run_5/community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_0.6_k_10_mincomm_50.csv


 55%|████████████████████████████████████████████████████████████████████████                                                           | 17183/31250 [03:25<02:37, 89.58it/s, loss=1.34]

[Task 104] Training model using dot similarity for Run 6, mu 0.25, k 10 on cuda:0...

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████                          | 25076/31250 [05:01<02:13, 46.28it/s, loss=1.31]

 33%|███████████████████████████████████████████▊                                                                                       | 10465/31250 [01:57<03:54, 88.80it/s, loss=1.02]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 22%|████████████████████████████▋                                                                                                       | 6797/31250 [01:20<05:42, 71.29it/s, loss=1.34]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit

In [4]:
m

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9,
 0.95,
 1.0]